In [18]:
import pandas as pd
import numpy as np

In [19]:
data = pd.read_csv("./data/train_dataset.csv")

data['is_test'] = False
evaluation = pd.read_csv("./data/evaluation_public.csv")
evaluation['is_test'] = True
sample = pd.read_csv("./data/sample_submission.csv")

all_data = pd.concat([data, evaluation]).reset_index(drop=True)

# 特征处理

In [20]:
all_data['date'] = pd.to_datetime(all_data['time'])
all_data['hour'] = all_data['date'].dt.hour
all_data['year'] = all_data['date'].dt.year
all_data['month'] = all_data['date'].dt.month
all_data['minute'] = all_data['date'].dt.minute
all_data['weekday'] = all_data['date'].dt.weekday
all_data['day'] = all_data['date'].dt.day
all_data['hour'] = all_data['date'].dt.hour
all_data['ts'] = all_data['hour']*3600 + all_data['minute']*60 + all_data['date'].dt.second

In [21]:
import gc
features = [ 'JS_NH3', 'CS_NH3', 'JS_TN', 'CS_TN', 'JS_LL', 'CS_LL', 'MCCS_NH4', 'MCCS_NO3', 'JS_COD', 'CS_COD', 'JS_SW', 'CS_SW', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL', 'B_QY_ORP', 'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_JS_DO', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP','weekday','hour', 'ts']
features = [f for f in features if f not in ['time', 'Label1', 'Label2','CS_LL','CS_NH3', 'JS_SW']] #




labels = ['Label1', 'Label2']
train = all_data[~all_data['is_test']].copy(deep=True)
test = all_data[all_data['is_test']].copy(deep=True)
test['is_train'] = False
train['is_train'] = True
data = pd.concat([train, test]).reset_index(drop=True)
del all_data
gc.collect()

for f in features:
    data[f] = data[f].fillna(method='ffill')

# 提取特征的diff值

In [22]:
# tmp_features = list(features)
# add_dict = {}
# for f in tmp_features:
#     data[f'{f}_diff']  = data[f].shift()
#     features.append(f'{f}_diff')


In [23]:
# fff = ['MCCS_NO3', 'B_HYC_NH4','N_HYC_NH4']
# for f in features:
#     data[f] = data[f].fillna(method='ffill')
# for i in range(len(fff)-1):
#     for j in range(i+1, len(fff)):
#         f1 = features[i]
#         f2 = features[j]
#         # feature_jian = f"{f1}_{f2}_jian"
#         feature_radio = f"{f1}_{f2}_radio"
#         data[feature_radio] = data[f1]/data[f2]
#         # data[feature_jian] = data[f1]-data[f2]
#         features.append(feature_radio)
#         # features.append(feature_jian)

In [24]:
df_list = []
i = 0
add_featuers = []
length = 0
for f in ['JS_NH3', 'CS_TN', 'JS_LL', 'JS_COD', 'CS_COD', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL',
          'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP']:
    for r in [15]:
        train_rolling = data[f].rolling(window=r, center=False)
        f_mean_name = 'rolling{}_{}_mean'.format(r,f)
        f_max_name = 'rolling{}_{}_max'.format(r,f)
        f_min_name = 'rolling{}_{}_min'.format(r,f)
        f_std_name = 'rolling{}_{}_std'.format(r,f)
        f_corr_name = 'rolling{}_{}_corr'.format(r,f)
        data[f_mean_name] = train_rolling.mean().fillna(0).values
        data[f_max_name] = train_rolling.max().fillna(0).values
        data[f_min_name] = train_rolling.min().fillna(0).values
        data[f_corr_name] = train_rolling.corr().fillna(0).values
        data[f_std_name] = train_rolling.std().fillna(0).values
        data[f'{f}_{f_mean_name}_cha'] = data[f] - data[f_mean_name]
        data[f'{f}_{f_max_name}_cha'] = data[f] - data[f_max_name]
        data[f'{f}_{f_min_name}_cha'] = data[f] - data[f_min_name]

        if i == 0:
            add_featuers.append(f_mean_name)
            add_featuers.append(f_max_name)
            add_featuers.append(f_min_name)
            add_featuers.append(f_std_name)
            add_featuers.append(f_corr_name)
            add_featuers.append(f'{f}_{f_mean_name}_cha')
            add_featuers.append(f'{f}_{f_max_name}_cha')
            add_featuers.append(f'{f}_{f_min_name}_cha')

    # for r in [5]:
    #     train_rolling = data[f].shift(15).rolling(window=r, center=False)
    #     f_mean_name = 'shift_rolling{}_{}_mean'.format(r,f)
    #     f_max_name = 'shift_rolling{}_{}_max'.format(r,f)
    #     f_min_name = 'shift_rolling{}_{}_min'.format(r,f)
    #     f_std_name = 'shift_rolling{}_{}_std'.format(r,f)
    #     data[f_mean_name] = train_rolling.mean().fillna(0).values
    #     data[f_max_name] = train_rolling.max().fillna(0).values
    #     data[f_min_name] = train_rolling.min().fillna(0).values
    #     data[f_std_name] = train_rolling.std().fillna(0).values
    #     data[f'shift_{f}_{f_mean_name}_cha'] = data[f] - data[f_mean_name]
    #     data[f'shift_{f}_{f_max_name}_cha'] = data[f] - data[f_max_name]
    #     data[f'shift_{f}_{f_min_name}_cha'] = data[f] - data[f_min_name]
    #
    #     if i == 0:
    #         add_featuers.append(f_mean_name)
    #         add_featuers.append(f_max_name)
    #         add_featuers.append(f_min_name)
    #         add_featuers.append(f_std_name)
    #         add_featuers.append(f'shift_{f}_{f_mean_name}_cha')
    #         add_featuers.append(f'shift_{f}_{f_max_name}_cha')
    #         add_featuers.append(f'shift_{f}_{f_min_name}_cha')
features.extend(add_featuers)

# 对所有的特征进行划分
for f in features:
    if f not in ['weekday','hour', 'ts']:
        q = len(data[f].drop_duplicates())
        data[f] = pd.qcut(data[f], q=int(q/10), labels=False, duplicates="drop")

train = data[data['is_train']].reset_index(drop=True)
test = data[~data['is_train']].reset_index(drop=True)

C:\Users\38125\AppData\Local\Temp/ipykernel_2456/2090275524.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{f}_{f_min_name}_cha'] = data[f] - data[f_min_name]
C:\Users\38125\AppData\Local\Temp/ipykernel_2456/2090275524.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f_mean_name] = train_rolling.mean().fillna(0).values
C:\Users\38125\AppData\Local\Temp/ipykernel_2456/2090275524.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whic

In [25]:
train

,time,JS_NH3,CS_NH3,JS_TN,CS_TN,JS_LL,CS_LL,MCCS_NH4,MCCS_NO3,JS_COD,...,N_CS_MQ_SSLL_rolling15_N_CS_MQ_SSLL_max_cha,N_CS_MQ_SSLL_rolling15_N_CS_MQ_SSLL_min_cha,rolling15_N_QY_ORP_mean,rolling15_N_QY_ORP_max,rolling15_N_QY_ORP_min,rolling15_N_QY_ORP_corr,rolling15_N_QY_ORP_std,N_QY_ORP_rolling15_N_QY_ORP_mean_cha,N_QY_ORP_rolling15_N_QY_ORP_max_cha,N_QY_ORP_rolling15_N_QY_ORP_min_cha
0,2022/1/4 0:00,237,0.0583,146,162,6902,996.0668,113,144,117,...,141,140,59,1,2,25,0,4804,100,86
1,2022/1/4 0:02,237,0.0583,146,162,7154,874.0959,113,153,117,...,141,140,59,1,2,25,0,4804,100,86
2,2022/1/4 0:04,254,0.0583,146,162,7111,1324.8105,111,148,117,...,141,140,59,1,2,25,0,4804,100,86
3,2022/1/4 0:06,254,0.0583,146,162,6804,723.9781,111,136,117,...,141,140,59,1,2,25,0,4804,100,86
4,2022/1/4 0:08,254,0.0583,146,162,6627,741.2993,111,129,117,...,141,140,59,1,2,25,0,4804,100,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140475,2022/7/18 2:30,166,0.0169,73,69,1789,1111.9569,76,253,74,...,31,78,5069,111,118,8169,4639,973,49,0
140476,2022/7/18 2:32,166,0.0169,73,69,1970,1680.4688,75,256,74,...,45,23,5048,111,117,7465,5695,703,39,0
140477,2022/7/18 2:34,166,0.0169,73,69,1726,1633.5309,75,255,74,...,98,72,5028,111,116,6949,6514,608,35,0
140478,2022/7/18 2:36,166,0.0169,73,69,1694,1247.8890,75,249,74,...,45,23,5008,110,116,6676,7104,533,32,0


# 删除分布不均衡的特征

In [26]:
train = train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)
def transform(x: pd.Series, c=20):
    return np.log1p(x/c)

def inverse_transform(x: pd.Series, c = 20):
    return np.expm1(x)*c

train = train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)
train['Label1'] = transform(train['Label1'])
label_c = 8
train['Label2'] = transform(train['Label2'], c=label_c)

In [27]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
score_list = []
for label in labels:
    train[label] = train[label].fillna(0)
    tmp = train[~train[label].isna()].reset_index(drop=True).copy(deep=True)
    print(len(tmp))
    print(np.max(tmp[label]), np.min(tmp[label]))
    test_size = 4000
    X_train = tmp[features][:-test_size]
    X_test = tmp[features][-test_size:]
    y_train = tmp[label][:-test_size]
    y_test = tmp[label][-test_size:]
    if label == "Label1":
        model = lgb.LGBMRegressor(
            boosting="gbdt",
            max_depth=4,
            learning_rate=0.1,
            n_estimators=10000,
            min_data_in_leaf=50,
            subsample = 0.8,
            feature_fraction=0.3,
            bagging_seed=1,
            reg_alpha=1,
            reg_lambda=1,  # 此处不改了
            min_sum_hessian_in_leaf=0.01,
            random_state=1212
        )
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['mse'],
              early_stopping_rounds=150, verbose=200)
        test_pred = model.predict(X_test)
        score_list.append(np.sqrt(mean_squared_error(inverse_transform(y_test), inverse_transform(test_pred))))
        print(f"score_list = {score_list}")
        model2 = lgb.LGBMRegressor(
            boosting="gbdt",
            max_depth=4,
            learning_rate=0.1,
            n_estimators=int(1.2*model.best_iteration_),
            min_data_in_leaf=50,
            subsample = 0.8,
            feature_fraction=0.3,
            bagging_seed=1,
            reg_alpha=1,
            reg_lambda=1,  # 此处不改了
            min_sum_hessian_in_leaf=0.01,
            random_state=1212
        )
        model2.fit(tmp[features], tmp[label])
        test[label] = inverse_transform(model2.predict(test[features]))
    else:
        model = lgb.LGBMRegressor(
            boosting="gbdt",
            max_depth=4,
            learning_rate=0.01,
            n_estimators=10000,
            min_child_weight=1,
            min_data_in_leaf=40,
            subsample = 0.4,
            feature_fraction=0.3,
            bagging_seed=11,
            reg_alpha=1,
            reg_lambda=1,  # 此处不改了
            min_sum_hessian_in_leaf=0.01,
            random_state=222
        )

        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['mse'],
              early_stopping_rounds=150, verbose=200)
        test_pred = model.predict(X_test)
        score_list.append(np.sqrt(mean_squared_error(inverse_transform(y_test, c=label_c), inverse_transform(test_pred, c=label_c))))
        print(f"score_list = {score_list}")
        model2 = lgb.LGBMRegressor(
            boosting="gbdt",
            max_depth=4,
            learning_rate=0.01,
            n_estimators=int(1.2*model.best_iteration_),
            min_child_weight=1,
            min_data_in_leaf=40,
            subsample = 0.4,
            feature_fraction=0.3,
            bagging_seed=11,
            reg_alpha=1,
            reg_lambda=1,  # 此处不改了
            min_sum_hessian_in_leaf=0.01,
            random_state=222
        )
        model2.fit(tmp[features], tmp[label])

        test[label] = inverse_transform(model2.predict(test[features]), c=label_c)

35068
8.106907231653508 3.731088486252612
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.3, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.01, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.01
Training until validation scores don't improve for 150 rounds
[200]	training's l2: 0.0074513	valid_1's l2: 0.0102004
[400]	training's l2: 0.00553489	valid_1's l2: 0.0099458
[600]	training's l2: 0.00441363	valid_1's l2: 0.010009
Early stopping, best iteration is:
[477]	training's l2: 0.00504996	valid_1's l2: 0.00991754
score_list = [1299.6478409581484]
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


In [28]:
loss = np.mean(score_list)
score = 1000/(1+loss)
print(score)

0.8343571481624228


In [29]:
test[['time'] + labels].to_csv(f"./res/lightgbm_res.csv", index=False)